# Decem libra historium

__author__ = ["Clément Besnier <clem@clementbesnier.fr>", ]

In [2]:
# Grégoire de Tours of Tours
from typing import List, Dict
from chr2020 import PACKDIR
n_books = 10

dhl_links = ["https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/Pr%C3%A9face", ]
dlh_link_pattern = "https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/"
dhl_links.extend([f"{dlh_link_pattern}{i}" for i in range(1, n_books+1)])

print(dhl_links)

['https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/Pr%C3%A9face', 'https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/1', 'https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/2', 'https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/3', 'https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/4', 'https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/5', 'https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/6', 'https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/7', 'https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/8', 'https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/9', 'https://fr.wikisource.org/wiki/Histoires_(Gr%C3%A9goire_de_Tours)/10']


## *Decem libra historium* retrieval

In [5]:
import os
import time
import codecs
import requests


def retrieve_texts(links):
    """

    :param links:
    :return:
    """
    texts = {}
    for link in links:
        r = requests.get(link)
        time.sleep(1)
        texts[link] = r.content
    return texts



def save_texts(texts: Dict[str, bytes], directory: str):
    """

    :param texts:
    :param directory:
    :return:
    """

    if not os.path.exists(directory):
        os.mkdir(directory)
    for i, link in enumerate(texts):
        filename = os.path.join(directory, str(i+1)+".html")
        with codecs.open(filename, "w", encoding="utf-8") as f:
            f.write(texts[link].decode("utf-8"))


In [6]:
dhl_directory_html = "gregoire_de_tours_html"
tours_texts = retrieve_texts(dhl_links)
save_texts(tours_texts, dhl_directory_html)

## *Decem libra historium* web to text

In [7]:
from bs4 import BeautifulSoup

def extract_text_from_html(directory: str) -> list:
    """
    From HTML files to TXT files



    :param directory:
    :return:
    """
    retrieved_texts = []
    filenames = sorted(os.listdir(directory), key=lambda x: int(x.split(".")[0]))
    print(filenames)
    for filename in filenames:
        with codecs.open(os.path.join(directory, filename), "r", encoding="utf-8") as f:
            text = f.read()
            tree = BeautifulSoup(text, "lxml")
            div = tree.find("div", {"class": "mw-parser-output"})
            retrieved_texts.append(div.get_text())
    return retrieved_texts

In [9]:
def save_txt_reformat(dst_directory: str,
                      retrieved_texts: list):
    """
    Save TXT files

    :param dst_directory:
    :param retrieved_texts:
    :return:
    """
    if not os.path.exists(dst_directory):
        os.mkdir(dst_directory)

    for i, text in enumerate(retrieved_texts):
        filename = os.path.join(dst_directory, f"{i+1}.txt")
        extracted_text = text
        with codecs.open(filename, mode="w", encoding="utf-8") as f:
            # lines = ["\t".join(line) for line in extracted_text]
            # final_text = "\n".join(lines)
            f.write(extracted_text)

In [10]:
print(os.getcwd())
dhl_directory = "gregoire_de_tours"
tours_retrieved_texts = extract_text_from_html(os.path.join(PACKDIR, "dlh", dhl_directory+"_html"))
save_txt_reformat(os.path.join(PACKDIR, "dlh", dhl_directory+"_txt"), tours_retrieved_texts)

C:\Users\clems\PycharmProjects\CHR2020\chr2020\dlh
['1.html', '2.html', '3.html', '4.html', '5.html', '6.html', '7.html', '8.html', '9.html', '10.html', '11.html']


## Manual cleaning